In [1]:
import tensorflow as tf
import numpy as np

from konlpy.tag import Twitter
import pandas as pd
import tensorflow as tf
import enum
import os
import re
import json
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

from preprocessor import *

2024-07-30 21:12:04.224983: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-30 21:12:04.519568: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 21:12:04.519612: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 21:12:04.569868: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-30 21:12:04.754286: I tensorflow/core/platform/cpu_feature_guar

# Graph visualization

In [2]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string], '')
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

# Load Data set

In [3]:
DATA_IN_PATH = './data_set/'
DATA_OUT_PATH = './data_out/'
TRAIN_INPUTS = 'train_inputs.npy'
TRAIN_OUTPUTS = 'train_outputs.npy'
TRAIN_TARGETS = 'train_targets.npy'
DATA_CONFIGS = 'data_configs.json'

In [4]:
index_inputs = np.load(open(DATA_IN_PATH + TRAIN_INPUTS, 'rb'))
index_outputs = np.load(open(DATA_IN_PATH + TRAIN_OUTPUTS , 'rb'))
index_targets = np.load(open(DATA_IN_PATH + TRAIN_TARGETS , 'rb'))
prepro_configs = json.load(open(DATA_IN_PATH + DATA_CONFIGS, 'r'))

# Fix Random Seed

In [5]:
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)

# Model Setting

In [6]:
char2idx = prepro_configs['char2idx']
end_index = prepro_configs['end_symbol']
model_name = 'transformer'
vocab_size = prepro_configs['vocab_size']
BATCH_SIZE = 2
MAX_SEQUENCE = 25
EPOCHS = 30
VALID_SPLIT = 0.1

In [8]:
char2idx

{'<PAD>': 0,
 '<SOS>': 1,
 '<END>': 2,
 '<UNK>': 3,
 '밤샜어': 4,
 '생각나네': 5,
 '잘자': 6,
 '밤이네요': 7,
 '세달': 8,
 '사로잡는': 9,
 '새벽': 10,
 '챙김받고': 11,
 '좋겠는': 12,
 '쇼핑했더니': 13,
 '붙잡아도': 14,
 '만들어줄까': 15,
 '울렸으면': 16,
 '냉정해진다': 17,
 '휘성노래가': 18,
 '톡을': 19,
 '아니지요': 20,
 '실행해보세요': 21,
 '걱정임': 22,
 '차였어': 23,
 '봐달라고': 24,
 '나른하다': 25,
 '이야기해주세요': 26,
 '휴반년이라는': 27,
 '3개월만에': 28,
 '듣고': 29,
 '결혼하재': 30,
 '쓴만큼': 31,
 '오려나': 32,
 '마지막으로': 33,
 '위험하네요': 34,
 '이상형뿐일': 35,
 '썼다가': 36,
 '기적같은': 37,
 '따르나봐요': 38,
 '양해를': 39,
 '않은거예요': 40,
 '허전하신가봐요': 41,
 '배웠길': 42,
 '한살': 43,
 '방금다녀와서': 44,
 '이사를': 45,
 '사람에게': 46,
 '첫만남': 47,
 '확인사살': 48,
 '정리해야할': 49,
 '책상': 50,
 '어릴땐': 51,
 '자격증': 52,
 '잔인했던가': 53,
 '미팅에서': 54,
 '울어버려요': 55,
 '통장': 56,
 '면접': 57,
 '욕은': 58,
 '잊혀질거라': 59,
 '바라는것도': 60,
 '빠졌나봐요': 61,
 '생각들이': 62,
 '없답니다': 63,
 '투자한': 64,
 '기억안': 65,
 '행복': 66,
 '야식': 67,
 '택배': 68,
 '답답하겠네요': 69,
 '거랑': 70,
 '답답한게': 71,
 '맞춰보는': 72,
 '귀찮게해': 73,
 '배울': 74,
 '생각': 75,
 '모른': 76,
 '멀리하세요': 77,
 '연속이예요': 

In [9]:
end_index

'<END>'

In [10]:
vocab_size

20705

In [7]:
kargs = {'model_name': model_name,
         'num_layers': 2,
         'd_model': 512,
         'num_heads': 8,
         'dff': 2048,
         'input_vocab_size': vocab_size,
         'target_vocab_size': vocab_size,
         'maximum_position_encoding': MAX_SEQUENCE,
         'end_token_idx': char2idx[end_index],
         'rate': 0.1
        }

# Padding & Forward-masking

In [11]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # add extra dimensions to add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [16]:
def create_look_ahead_mask(size):
    # 마스킹할 영역을 1, 아닌 영역을 0으로 처리
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0) #(입력 행렬, -1, 0)
    return mask  # (seq_len, seq_len)

In [17]:
def create_masks(inp, tar):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(inp)

    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask

In [20]:
index_inputs

array([[11034,  4489,     0, ...,     0,     0,     0],
       [ 3278, 11766, 16612, ...,     0,     0,     0],
       [ 4584, 15569, 13314, ...,     0,     0,     0],
       ...,
       [ 1245,  4083, 19327, ...,     0,     0,     0],
       [17844, 11697,  8357, ...,     0,     0,     0],
       [ 9695, 12475,     0, ...,     0,     0,     0]], dtype=int32)

In [21]:
index_outputs

array([[    1, 10553,  2770, ...,     0,     0,     0],
       [    1, 11813, 10407, ...,     0,     0,     0],
       [    1, 10025, 12993, ...,     0,     0,     0],
       ...,
       [    1,  5504,     0, ...,     0,     0,     0],
       [    1, 11580,  8964, ...,     0,     0,     0],
       [    1, 17285,  2490, ...,     0,     0,     0]], dtype=int32)

In [18]:
enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(index_inputs, index_outputs)

2024-07-30 23:20:45.310372: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-30 23:20:45.499201: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-30 23:20:45.499242: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-30 23:20:45.504997: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-30 23:20:45.505073: I external/local_xla/xla/stream_executor

In [24]:
enc_padding_mask.shape

TensorShape([11823, 1, 1, 25])

In [23]:
look_ahead_mask.shape

TensorShape([11823, 1, 25, 25])

In [25]:
dec_padding_mask.shape

TensorShape([11823, 1, 1, 25])